In [1]:
import numpy as np
import time 
from sklearn.neighbors import NearestNeighbors
import math

Given any available load (`avl_load`) and list of charger wattages (`chargers`), we want to give an optimal distribution of the number of chargers of each type. We will be implimenting a `Maximum Inner Product Search` using `Nearest Neighbor` search algorithms.

- **Constraint 1** : *must have atleast `min_chargers` number of chargers of each type and the total number of chargers must not be more than `max_chargers`*.

In [16]:
def transform(vecs):
    maxnorm = max([np.linalg.norm(v) for v in vecs])
    new_vecs = []
    for v in vecs:
        new_vecs.append(np.insert(v, 0, np.sqrt(maxnorm**2-np.linalg.norm(v)**2)))
    return new_vecs

def mips_naive(q, vecs):
    mip = -1e10             # initial max mip. as we know our vectors are positive we can set it to zero for our case 
    idx = -1
    for i, v in enumerate(vecs):
        if np.dot(q, v) > mip:
            mip = np.dot(q, v)
            idx = i
    return idx, mip

def charger_dist(chargers, avl_load, min_chargers=1, max_chargers=100):
    low = min_chargers
    high = [math.ceil(avl_load/p) for p in chargers]
    if sum(high)>max_chargers:
        high = max_chargers

    num = math.prod(high)

    vecs = np.random.randint(low, high, (num,3))   # random initialization
    vecs_prod = list(np.array([np.array(chargers).dot(v) for v in list(vecs)]))
    vecs_indices = [i for i in range(len(vecs_prod)) if vecs_prod[i] <=avl_load]
    vecs_list = [vecs[i] for i in vecs_indices]

    maxnorm = max([np.linalg.norm(v) for v in vecs_list])
    vecs_trans = transform(vecs_list)

    q = np.array(chargers)
    q_trans = np.insert(q, 0, 0)

    X = np.array(vecs_trans)
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='kd_tree').fit(X)        # we can set the nearest neighbours to any number we want if we want a few closest answers

    distances, indices = nbrs.kneighbors(np.array([q_trans]))

    return list(map(lambda j: vecs_list[j], indices[0]))

In [17]:
avl_load = 230
chargers= [7, 23, 72]
charger_dist(chargers, avl_load)

[array([2, 6, 1])]